The main goal of this notebook is to use a LAS file to analyze the surface area covered by tree vegetation in forests in Spain. To achieve this, an unsupervised algorithm will be used to cluster the points in the LIDAR point cloud.

Load the LIDAR Data: Read the LAS file containing the point cloud data.

Filter Vegetation Points: Isolate the points representing tree vegetation based on their classification in the LIDAR dataset.

Cluster the Vegetation Points: Use an unsupervised clustering algorithm to group the points representing trees.

Analyze the Surface Area: Calculate the surface area covered by the clustered points to estimate the vegetation coverage.

In [2]:
#%% 1. Aerial Lidar Vectorization: Implementation Setup

#Base libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#3D Libraries
import open3d as o3d
import laspy
print(laspy.__version__)

#Geospatial libraries
import rasterio
import alphashape as ash
import geopandas as gpd
import shapely as sh

from rasterio.transform import from_origin
from rasterio.enums import Resampling
from rasterio.features import shapes
from shapely.geometry import Polygon


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
2.5.3


In [1]:
#Set the relative path
input_path = "./PNOA_files/PNOA_2019_CYL_NE_368-4740_000-000-RGB.laz"

In [3]:
# Open the LAS file specified by input_path
with laspy.open(input_path) as fh:
    
    # Print the number of points as recorded in the file header
    print('Points from Header:', fh.header.point_count)
    
    # Read the full data from the LAS file into a laspy object
    las = fh.read()
    
    # Print the entire laspy object (contains point cloud data and metadata)
    print(las)
    
    # Print the actual number of points in the dataset
    print('Points from data:', len(las.points))
    
    # Create a mask for ground points (classification code 2 is ground)
    ground_pts = las.classification == 2
    
    # Get unique return numbers for ground points and their counts
    bins, counts = np.unique(las.return_number[ground_pts], return_counts=True)
    
    # Print the distribution of return numbers for ground points
    print('Ground Point Return Number distribution:')
    
    # Loop through the unique return numbers and their counts, and print them
    for r, c in zip(bins, counts):
        print('    {}:{}'.format(r, c))

Points from Header: 28226896
<LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 28226896 points, 3 vlrs)>
Points from data: 28226896
Ground Point Return Number distribution:
    1:7149780
    2:1484452
    3:62308
    4:23199
    5:6426
    6:1198
    7:159


In [5]:
import laspy
import numpy as np

# Read the point cloud data from the LAS file specified by input_path
point_cloud = laspy.read(input_path)

# Extract the Z (altitude) values from the point cloud and organize into a vertical stack
pointsZ = np.vstack((point_cloud.z)).transpose()

# Print the maximum altitude
print('\nMaximum altitude:', np.max(pointsZ))

# Print the average (mean) altitude
print('\nAverage altitude:', np.mean(pointsZ))

# Print the minimum altitude
print('\nMinimum altitude:', np.min(pointsZ))



Altitud maxima: 1322.795

Alitud media: 1126.8697595862131

Altitud minima: 962.172


In [30]:
#%% 3. Data Preprocessing

#%% 3.1. Initialization of Building Points
# Create a mask to filter points classified as buildings
pts_mask = las.classification == 4

# Apply the mask and obtain the coordinates from the dataset (x, y, z)
xyz_t = np.vstack((las.x[pts_mask], 
                   las.y[pts_mask], 
                   las.z[pts_mask]))

# Transform the filtered points into a 3D geometry (point cloud)
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(xyz_t.transpose())

# Compute the center of the point cloud and store the translation for later use
pcd_center = pcd_o3d.get_center()
#pcd_o3d.translate(-pcd_center)  # Uncomment to translate the point cloud to the origin

# Visualize the resulting point cloud
o3d.visualization.draw_geometries([pcd_o3d])


In [31]:
#%% 3.2. Isolation of Ground Points
# Create a mask to filter points classified as ground (classification code 2)
pts_mask = las.classification == 2

# Apply the mask and obtain the x, y, z coordinates of the ground points
xyz_t = np.vstack((las.x[pts_mask], las.y[pts_mask], las.z[pts_mask]))

# Create a PointCloud object to store the ground points
ground_pts = o3d.geometry.PointCloud()
ground_pts.points = o3d.utility.Vector3dVector(xyz_t.transpose())

# Optionally, translate the ground points to the same center as the previous point cloud
# ground_pts.translate(-pcd_center)  # Uncomment to apply the translation

# Visualize the ground points
o3d.visualization.draw_geometries([ground_pts])


In [32]:
#%% 3.2. Identifying the average distance between building points
nn_distance = np.mean(pcd_o3d.compute_nearest_neighbor_distance())
print("Average point distance (m): ", nn_distance)

average point distance (m):  0.5208210311347834


In [43]:
#%% 4. Unsupervised Segmentation (Clustering) with DBSCAN
# We are now working in a local reference frame, which is essential to avoid issues with truncated coordinates

# Define the parameters: epsilon (neighborhood size) and the minimum number of points to consider a relevant cluster
epsilon = 3
min_cluster_points = 25

# Perform DBSCAN clustering
labels = np.array(pcd_o3d.cluster_dbscan(eps=epsilon, min_points=min_cluster_points))
max_label = labels.max()
print(f"The point cloud contains {max_label + 1} clusters")

# Use a discrete color palette to randomize the cluster visualization
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # points with no assigned cluster are colored black
pcd_o3d.colors = o3d.utility.Vector3dVector(colors[:, :3])

# Visualize the point cloud with the clusters
o3d.visualization.draw_geometries([pcd_o3d])


The point cloud contains 1057 clusters


In [44]:
#%% 6. Extracting the outline (building footprint) of the selection

building_vector = []
#We extract only the X and Y coordinates of our point cloud (Note: it is local)

for i in range(max_label + 1):

    sel = i
    segment = pcd_o3d.select_by_index(np.where(labels==sel)[0])
    #o3d.visualization.draw_geometries([segment])

    points_2D = np.asarray(segment.points)[:,0:2]

    #We compute the shape (alpha shape) and return the result with shapely
    building_vector.append(ash.alphashape(points_2D, alpha=0.5))
    building_vector

In [45]:
# Set the geospatial data from the coordinate reference system EPSG:25830 (ETRS89 / UTM zone 30N)
building_gdf = gpd.GeoDataFrame(geometry=building_vector, crs='EPSG:25830')
building_gdf

,geometry
0,"POLYGON ((369735.990 4738011.840, 369736.420 4..."
1,"MULTIPOLYGON (((369783.290 4738226.240, 369780..."
2,"POLYGON ((369999.410 4738010.890, 369999.460 4..."
3,"POLYGON ((368128.960 4738010.970, 368128.980 4..."
4,"POLYGON ((368098.560 4738004.020, 368099.370 4..."
...,...
1052,"POLYGON ((369892.070 4739937.870, 369891.630 4..."
1053,"POLYGON ((369828.890 4739954.290, 369828.430 4..."
1054,"POLYGON ((369818.000 4739960.810, 369817.200 4..."
1055,"POLYGON ((369888.640 4739968.770, 369888.130 4..."


In [46]:
# Save the shape
output_path = "./output/forest_shapefile2.shp"
building_gdf.to_file(output_path, driver='ESRI Shapefile')